# OpenAI-Translator 进阶版本（v2.0）开发详解

## 介绍

Tip:这里是作业，如果是路人的话[点击这里](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/README-CN.md)进入正式项目的ReadMe

在学习[极客时间](https://time.geekbang.org/)的[《AI 大模型应用开发实战营》](https://u.geekbang.org/subject/llm/1005515)的过程中，我用GPT-4对OpenAI-Translator进行了进一步的升级，形成了v2.0版本。在该文档中，我将详细介绍本版本中的三个新功能：

1. 用户友好的GUI界面设计。
2. 多语言选择的翻译支持。
3. 对于pdf中的表格和文本，v2.0能够更好地保留其原始布局和格式。


---
| v1.0 功能                                      | v2.0 更新                                |
|:-----------------------------------------------|:----------------------------------------|
| [X] 使用大型语言模型 (LLMs) 将英文 PDF 书籍翻译成中文。 | [X] 无变化                               |
| [X] 支持 ChatGLM 和 OpenAI 模型。              | [X] 无变化                               |
| [X] 通过 YAML 文件或命令行参数灵活配置。       | [X] 无变化                               |
| [X] 对健壮的翻译操作进行超时和错误处理。        | [X] 无变化                               |
| [X] 模块化和面向对象的设计，易于定制和扩展。   | [X] 无变化                               |
| [ ] 实现图形用户界面 (GUI) 以便更易于使用。     | [X] 已实现                              |
| [ ] 添加对多个 PDF 文件的批处理支持。           | [ ] 未实现                              |
| [ ] 创建一个网络服务或 API，以便在网络应用中使用。 | [ ] 未实现                              |
| [ ] 添加对其他语言和翻译方向的支持。            | [X] 已实现                              |
| [ ] 添加对保留源 PDF 的原始布局和格式的支持。    | [X] 已实现                              |
| [ ] 通过使用自定义训练的翻译模型来提高翻译质量。 | [ ] 未实现                              |

---

上表展示了已完成的工作进度与待完成项


### 1. 用户友好的GUI界面设计



开发这个进阶版本的时候，一个重要的方面就是提供一个用户友好的图形界面。这里，主要介绍两个方面的改进：命令行参数的扩展以及GUI模块的实现。

####  命令行参数的改动

首先，我对命令行参数进行了扩展，为GUI界面提供了支持。具体地，新增了一个`gui`参数，同时对`check_argument`函数进行了适当扩展。

以下是相关代码片段：

```python
def __init__(self):
    ...
    self.parser.add_argument('--gui', action='store_true', help='Start GUI mode')

def parse_arguments(self):
    return self.parser.parse_args()

    def check_argument(self, args):
        if args.model_type == 'OpenAIModel':
            if not args.openai_model:
                self.parser.error("--openai_model is required when using OpenAIModel")
            if not args.openai_api_key:
                self.parser.error("--openai_api_key is required when using OpenAIModel")
            if not args.book:
                self.parser.error("--book is required")
            if not args.file_format:
                self.parser.error("--file_format is required")


这样的设计使得命令行界面更加强大和灵活，用户可以根据自己的需求选择是否启动GUI模式。

#### GUI模块的设计实现

在设计GUI模块时，我使用了Gradio库，它是一个非常直观且强大的Python库，用于快速创建交互式UI。

在windows上，可以在控制台输入以下指令运行GUI：

```bash
python ai_translator/main.py --model_type OpenAIModel --gui
```

默认情况下，gradio会在http://127.0.0.1:7860 上运行gui。可以通过浏览器打开

下图展示了GUI的界面，使用者可以拖动文件或从文件浏览器上传pdf文档。然后选择目标语言，和输出文件格式。翻译成功后将在输出栏展示输出后的文件地址。

![GUI界面](images/gui.png)

---

### 2. 多语言选择的翻译支持

为了更好地满足各种语言环境的用户需求，我在OpenAI-Translator中加入了对多种语言的支持。下面我们将简要介绍如何通过GUI界面扩展支持的目标语言以及如何使用system prompt来更好地引导模型进行翻译。

#### 扩展GUI界面支持的语言

目前，GUI界面已支持中文、日语和西班牙语。要增加更多的语言支持，您可以修改[gui_interface.py](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/ai_translator/gui/gui_interface.py)文件中的两个部分：

i. 语言映射：您可以在language_mapping字典中添加新的语言对应关系。例如，要增加法语，您可以这样做：

```python
language_mapping = {
    "中文": "Chinese",
    "日语": "Japanese",
    "西班牙语": "Spanish",
    "法语": "French" # 新增的语言映射
}
```

ii. 下拉菜单选择：
您还需要在下拉菜单的choices参数中添加新的语言选项，这样用户就可以在GUI界面中选择它了。

```python
gr.inputs.Dropdown(choices=["中文", "日语", "西班牙语", "法语"], default="中文", label="选择目标语言")
```

#### 使用system prompt优化翻译效果
当涉及到多语言翻译时，我们需要确保翻译的准确性和一致性。为了更好地指导模型进行翻译，我们使用了system prompt和few-shot learning的手段。另外，为了保证准确性，代码中[prompt template](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/ai_translator/model/prompt_template.py)使用了英语版本。以下是这个提示的中文翻译：

##### System Prompt:

---

您是一个先进的翻译助手，擅长在各种语言之间进行翻译。您的任务是：

- 准确地检测源语言。
- 尽可能精确地翻译内容。
- 保留原始的格式，如表格、空格、标点符号和特殊结构。
以下是一些指导您的示例：

##### 示例1:
文本: 'Hello, how are you?'

翻译（到西班牙语）: Hola, ¿cómo estás?

##### 示例2:
文本: 'Je suis heureux.'

翻译（到日语）: 私は幸せです。

注意：对于文本内容，请准确翻译内容，不要添加或删除任何标点符号或符号。

##### 示例3:
表格: '[Name, Age] [John, 25] [Anna, 30]'

翻译（到中文）: [姓名, 年龄] [约翰, 25] [安娜, 30]

注意：对于表格内容，使用方括号和逗号作为分隔符保持格式。仅返回括在括号内的翻译内容，不要添加任何额外的解释。特别是当翻译成日语时，不要将逗号","翻译为日语标点"、"或"、"。

现在，根据指定的目标语言，继续进行翻译，检测源语言。

---

经过测试，目前该prompt可以良好的指导GPT进行准确的翻译，并且返回正确的表格格式。

但是，可以注意到prompt中对日语翻译时，对标点符号有额外提示，这是因为，经过测试发现GPT-3.5容易将表格中的分隔符逗号翻译为顿号。这意味着：对于扩展其他语言的翻译任务，可能也需要通过测试对prompt继续进行迭代优化。

---

### 3. 对于pdf中的表格和文本，v2.0能够更好地保留其原始布局和样式。

#### 对于文本布局的处理：

1. **数据结构更新**：
   根据`content`基类新增了一个数据结构`paragraph`用于存储段落。并且为`content`新增了`layout`属性。  
   *代码示例:*  

```python
   class Paragraph:
       def __init__(self, text, layout=None, translation=None, style=None):
           self.text = text
           self.layout = layout
           self.tanslation = translation
           self.style = style if style else {}
```

2. **解析PDF时的段落处理**：
    在解析pdf时，将根据上下文行间距分割段落，并且获取每一段的顶部坐标`top`和底部坐标`bottom`存入`paragraph`的属性`layout`。
    *代码示例*:
    
```python
 # Group words by top value to detect lines
                lines = {}
                for word in words:
                    lines.setdefault(word['top'], []).append(word)

                grouped_lines = sorted(lines.values(), key=lambda l: l[0]['top'])

                # Construct paragraphs based on empty space or large gaps between lines
                paragraphs = []
                current_paragraph = []
                previous_bottom = 0
                for line_words in grouped_lines:
                    if previous_bottom and (line_words[0]['top'] - previous_bottom) > 10:  # Assuming gap > 10 means new paragraph
                        paragraphs.append(current_paragraph)
                        current_paragraph = []

                    current_paragraph.extend(line_words)
                    previous_bottom = line_words[0]['bottom']
                    
                    ...
                    
                    layout = {
                        'top': paragraph_words[0]['top'],
                        'bottom': paragraph_words[-1]['bottom']
                    }
                    paragraph = Paragraph(text=paragraph_text, layout=layout, style=style)
                    
```

3. **合并段落，整页文本一起翻译**：为了减少API调用次数(每1分钟只能调用3次)，段落被合并成了文本一起传递给API。为了使翻译后的文本可以成功分段，使用了添加标识符的方法。
    *代码示例*:
    
```python
# 将每个段落添加到original属性，并插入标识符
                    if content.original:
                        content.original += '\n' + "¶¶¶" + '\n'
                    content.original += paragraph_text               
```

4. **pdf排版，生成文本**：在[writer.py](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/ai_translator/translator/writer.py)中可以根据`paragraph`先前存入的`layout`顺序，顺序排版每一段文字。

#### 对于table的处理：

1. **Table的Layout处理**：
`table`也存储了`layout`属性，writer时会比较`paragraph`与`table`的行位置，在合适的位置插入图表，保证图表与段落的相对位置。除此以外，table的layout还扩展了`left`和`right`属性，在`writer`时会复原表格原本的宽度。

实际实现流程与paragraph类似，不再赘述。

2. **插图说明**：
下图展示了在段落中存在表格时，表格的布局情况。

![test1](images/test1.png)

#### 对于文本样式style的处理：
Content和Paragraph扩展了style属性。但是目前只简单实现了保留了字体大小。字体和加粗，斜体，颜色等样式信息还没有实现。
```python

 #   这个方法没有完全实现，只简单估计了文本大小
    def extract_style_from_word(self, word):
         # Use 'top' and 'bottom' to estimate the font size
        font_height = word['bottom'] - word['top']
        
        style = {
            "size": font_height,  # Use the calculated font height as size
            "font": word.get('font', None),  # Example to extract font. Adjust based on Word object's attributes
            # You may need more intricate logic to determine 'bold' and 'italy' properties based on font names or other attributes.
            "bold": "Bold" in word.get('font', ''),
            "italy": "Ital" in word.get('font', '') or "Italic" in word.get('font', '')
        }
        return style
```

下图展示了对于原文字体大小的保留，可以看出标题，正文与表格描述三种文本具有不同的字体大小：

![test2](images/test2.png)

## 测试

我们使用了以下四个测试文档来验证翻译和布局处理功能：

1. **test2.pdf**: 该文档主要用于测试文字大小的保留情况。
   - [查看原文档](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/test2.pdf)
   - [查看中文翻译结果](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/test2_translated.pdf)
   - [查看日语翻译结果](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/test2_translated_Japanese.pdf)
   - [查看西班牙语翻译结果](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/test2_translated_Spanish.pdf)

2. **twotable.pdf**: 该文档包含两个表格，用于测试如何处理多个表格的情况。
   - [查看原文档](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/twotable.pdf)
   - [查看翻译结果](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/twotable_translated.pdf)

3. **2 pages.pdf**: 这是一个两页的文档，包含三个表格。它提供了一个更全面的测试案例，以确保所有功能都能正常工作。
   - [查看原文档](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/2%20pages.pdf)
   - [查看翻译结果](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/2%20pages_translated.pdf)

4. **RussiaToChinese.pdf**: 该文档是用于测试逆向转换的，即将PDF中的俄语转换为中文。
   - [查看原文档](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/RussiaToChinese.pdf)
   - [查看翻译结果](https://github.com/carrotProgrammer/OPENAI_TRANSLATOR/blob/master/jupyter/RussiaToChinese_translated.pdf)

在测试中，前三份英文文档都成功地被翻译成了日语、中文和西班牙语。而最后一份文档，也实现了从俄语到中文的准确翻译。整体来看，翻译后的文档布局与原文相近。但值得注意的是，在“2 pages.pdf”文档中，为了维持小表格的显示效果，翻译文本超出了文本框范围。这是接下来需要优化的问题。



## 当前版本限制

鉴于PDF格式的固有复杂性，尽管已努力优化，但本工具在处理某些PDF内容时仍存在限制：

- **翻译准确性**：尽管OpenAI翻译器基于GPT模型，拥有强大的翻译能力。但不能保证对所有语言都能达到100%的翻译准确性。
  
- **格式一致性**：虽然翻译后的文档试图尽量保持与原文档相似，但依然存在差异，主要体现在字体样式与表格样式的差异上。

- **图像支持**：当前版本不支持PDF中的图像翻译。

- **扫描文档支持**：经过扫描的PDF文件可能无法被正确识别和翻译。

作为用户，应当明确这是一个工具，虽然它在多数场景下都非常有用，但并不能完全替代专业的翻译软件或服务。


## 学习心得（其实也是GPT生成的）

经过《AI大模型课程》的深入学习，我对大型AI模型有了更全面的认识。课程内容从基础原理如注意力机制、transformer等，到实际应用的API接口，再到翻译软件的开发，为我铺设了一条通向AI的宽阔大道。这不仅使我对AI的理论有了更深入的理解，更让我积累了丰富的实践经验。

在实现 OpenAI-Translation 的过程中，我深深体会到了理论与实践的紧密结合。面对各种挑战，我并未退缩，而是通过与其他开发者的交流、查阅相关文档、及时的测试与调试，逐步优化和完善了我的翻译工具。

在此，我特别要感谢彭老师与教学组的老师们。他们的指导和支持是我在学习和实践过程中的重要后盾。

虽然我为目前取得的成果感到骄傲，但我深知学习是一个持续的过程。《AI大模型课程》为我提供了宝贵的学习机会，而OpenAI则为我提供了强大的技术支撑。眼下，这只是一个阶段性的成果，未来，我仍期望更进一步，挖掘AI更多的可能性。